In [133]:
from __future__ import  division
import numpy as np
from copy import copy, deepcopy
import matplotlib.pyplot as plt
import math
from mpl_toolkits.mplot3d import Axes3D
from scipy import linalg, special
from numpy.linalg import norm
import statsmodels.api as st
import family
from scipy.sparse import linalg as sp_linalg
from sklearn import datasets, neighbors, linear_model
from sklearn.preprocessing import LabelBinarizer
from optimize import newton_cg
from scipy import optimize
from scipy.misc import logsumexp
import statsmodels.regression.linear_model as lim
import pandas as pd
import warnings
import datetime
import pytz
from datetime import timedelta
from collections import Counter
from scipy.stats import gaussian_kde
import glob, os
import warnings
import statsmodels.api as sm
import seaborn as sns
import warnings
warnings.simplefilter("ignore")

%matplotlib inline

In [134]:
def _rescale_data(X, Y, sample_weight):
    """Rescale data so as to support sample_weight"""
    n_samples = X.shape[0]
    sqrtW = np.sqrt(sample_weight)
    
    newX = X * sqrtW.reshape(-1,1)
    newY = Y * sqrtW.reshape(-1,1)
    
    return newX, newY

In [135]:
def addIntercept(X):
    t = X.shape[0]
    X_with_bias = np.hstack((np.ones((t, 1)), X))
    return X_with_bias

In [136]:
class BaseModel(object):
    """
    A generic supervised model for data with input and output.
    BaseModel does nothing, but lays out the methods expected of any subclass.
    """

    def __init__(self, fam, solver, fit_intercept = True, penalty = None, reg = 0, l1_ratio = 0, tol = 1e-4, max_iter = 100):
        """
        Constructor
        Parameters
        ----------
        fam: family of the GLM, LM or MNL
        solver: family specific solver
        penalty: penalty to regularize the model
        reg: regularization strenth
        l1_ratio: if elastic net, the l1 reg ratio
        tol: tol in the optimization procedure
        max_iter: max_iter in the optimization procedure
        -------
        """
        self.fit_intercept = fit_intercept
        self.penalty = penalty
        self.reg = reg
        self.l1_ratio = l1_ratio
        self.fam = fam
        self.solver = solver
        self.tol = tol
        self.max_iter = max_iter

        

    def fit(self, X, Y, sample_weight = None):
        """
        fit the weighted model
        Parameters
        ----------
        X : design matrix
        Y : response matrix
        sample_weight: sample weight vector
        

        """
        raise NotImplementedError

    def predict(self, X):
        """
        predict the Y value based on the model
        ----------
        X : design matrix
        Returns
        -------
        predicted value
        """
        return NotImplementedError

    def probability(self, X, Y):
        """
        Given a set of X and Y, calculate the probability of
        observing Y value
        """
        logP = self.log_probability(X, Y)
        if logP is not None:
            return np.exp(self.log_probability(X, Y))
        else:
            return None
    
    def log_probability(self, X, Y):
        """
        Given a set of X and Y, calculate the log probability of
        observing each of Y value given each X value
        
        should return a vector
        """
        return NotImplementedError
    
    def estimate_dispersion(self):
        raise NotImplementedError
        
    def estimate_sd(self):
        raise NotImplementedError
    
    def estimate_loglikelihood(self):
        raise NotImplementedError
    

In [137]:
class GLM(BaseModel):
    """
    A Generalized linear model for data with input and output.
    """
    def __init__(self, fam, solver = 'pinv', fit_intercept = True, penalty = None, reg = 0, l1_ratio = 0, tol = 1e-4, max_iter = 100):
        super(GLM, self).__init__(fam = fam, solver = solver, fit_intercept = fit_intercept, penalty = penalty, 
                                 reg = reg, l1_ratio = l1_ratio, tol = tol, max_iter = max_iter)
    def fit(self, X, Y, sample_weight = None):
        """
        fit the weighted model
        Parameters
        ----------
        X : design matrix
        Y : response matrix
        sample_weight: sample weight vector

        """
         # family is the glm family with link, the family is the same as in the statsmodel
        

        if sample_weight is None:
            sample_weight = np.ones((X.shape[0],))
        assert X.shape[0] == sample_weight.shape[0]
        assert X.shape[0] == Y.shape[0]
        assert Y.ndim == 1 or Y.shape[1] == 1
        Y = Y.reshape(-1,)

        sum_w = np.sum(sample_weight)
        assert sum_w > 0
        
        
        if X.ndim == 1:
            X = X.reshape(-1,1)
        if self.fit_intercept:
            X = addIntercept(X)   
        self.n_samples = X.shape[0]
        self.n_features = X.shape[1]
        self.n_targets = 1


        # start fitting using irls
        mu = self.fam.starting_mu(Y)
        lin_pred = self.fam.predict(mu)
        dev = self.fam.deviance_weighted(Y, mu, sample_weight)

        if np.isnan(dev):
            raise ValueError("The first guess on the deviance function "
                             "returned a nan.  This could be a boundary "
                             " problem and should be reported.")

        # This special case is used to get the likelihood for a specific
        # params vector.

        for iteration in range(self.max_iter):
#             print sample_weight.shape
#             print self.fam.weights(mu)
            weights = sample_weight * self.fam.weights(mu)
            wlsendog = lin_pred + self.fam.link.deriv(mu) * (Y-mu)
            if self.penalty is None:
                wls_results = lim.WLS(wlsendog, X, weights).fit(method = self.solver)

            if self.penalty == 'elasticnet':
                wls_results = lim.WLS(wlsendog, X, weights).fit_regularized(alpha = self.reg, L1_wt = self.l1_ratio)


            lin_pred = np.dot(X, wls_results.params)
            mu = self.fam.fitted(lin_pred)

            if Y.squeeze().ndim == 1 and np.allclose(mu - Y, 0):
                msg = "Perfect separation detected, results not available"
                raise Error(msg)

            dev_new = self.fam.deviance_weighted(Y, mu, sample_weight)
            converged = np.fabs(dev - dev_new) <= self.tol
            dev = dev_new
            if converged:
                break
        
        self.converged = converged
        self.coef = wls_results.params
        self.dispersion = self.estimate_dispersion(X, Y, mu, sample_weight)
        self.sd = self.estimate_sd(X, Y, mu, sample_weight, weights)
        self.ll = self.estimate_loglikelihood(Y, mu, sample_weight)

    def predict(self, X):
        """
        predict the Y value based on the model
        ----------
        X : design matrix
        Returns
        -------
        predicted value
        """

        if X.ndim == 1:
            X = X.reshape(-1,1)
        if self.fit_intercept:
            X = addIntercept(X)
        lin_pred = np.dot(X, self.coef)
        mu = self.fam.fitted(lin_pred)
        return mu

    
    def log_probability(self, X, Y):
        """
        Given a set of X and Y, calculate the probability of
        observing Y value
        """

        mu = self.predict(X)
#         print mu.shape
#         print X.shape
#         print Y.shape
        return self.fam.log_probability(Y.reshape(-1,), mu, scale=self.dispersion)
    
    
    def estimate_dispersion(self, X, Y, mu, w):
        if isinstance(self.fam, (family.Binomial, family.Poisson)):
            return 1
        else:
            resid = (Y - mu)
            return (resid ** 2 * w / self.fam.variance(mu)).sum()/ np.sum(w)
    
    def estimate_sd(self, X, Y, mu, w, weights):
        if self.penalty is None and self.dispersion is not None:
            newX, newY = _rescale_data(X, Y, weights)
            wX, wY = _rescale_data(X, Y, w * weights)
            if X.shape[1] == 1:
                try:
                    cov = 1 / np.dot(newX.T, newX)    
                    temp = np.dot(wX.T, wX) 
                    sd = (np.sqrt(cov ** 2 * temp)  * np.sqrt(self.dispersion)).reshape(-1,)

                except:
                    sd = None         
            else:
                try:
                    cov = np.linalg.inv(np.dot(newX.T, newX))
                    temp = np.dot(cov, wX.T)
                    sd = np.sqrt(np.diag(np.dot(temp,temp.T))) * np.sqrt(self.dispersion)
                except:
                    sd = None
        else:
            sd = None
        return sd

    
    def estimate_loglikelihood(self, Y, mu, w):
        if self.dispersion is None:
            return None
        else:
            return self.fam.loglike_weighted(Y, mu, w, scale=self.dispersion)

In [138]:
class LM(BaseModel):
    """
    A Generalized linear model for data with input and output.
    """
    def __init__(self, solver = 'svd', fit_intercept = True, penalty = None, reg = 0, l1_ratio = 0, tol = 1e-4, max_iter = 100):
        super(LM, self).__init__(fam = 'LM', solver = solver, fit_intercept = fit_intercept, penalty = penalty, 
                                 reg = reg, l1_ratio = l1_ratio, tol = tol, max_iter = max_iter)
        
    def fit(self, X, Y, sample_weight = None):
        """
        fit the weighted model
        Parameters
        ----------
        X : design matrix
        Y : response matrix
        sample_weight: sample weight vector

        """

        if sample_weight is None:
            sample_weight = np.ones((X.shape[0],))
        assert X.shape[0] == sample_weight.shape[0]
        assert X.shape[0] == Y.shape[0]

        sum_w = np.sum(sample_weight)
        assert sum_w > 0
        if X.ndim == 1:
            X = X.reshape(-1,1)
        if self.fit_intercept:
            X = addIntercept(X)
        if Y.ndim == 1:
            Y = Y.reshape(-1,1)

        self.n_samples = X.shape[0]
        self.n_features = X.shape[1]
        self.n_targets = Y.shape[1]
        
        newX, newY = _rescale_data(X, Y, sample_weight)


        if self.penalty is None:
            model = linear_model.LinearRegression(fit_intercept=False)
        if self.penalty == 'l1':
            model = linear_model.Lasso(fit_intercept=False, alpha = self.reg, tol = self.tol, max_iter = self.max_iter)
        if self.penalty == 'l2':
            model = linear_model.Ridge(fit_intercept=False, alpha = self.reg, tol = self.tol, 
                                       max_iter = self.max_iter, solver = self.solver)
        if self.penalty == 'elasticnet':
            model = linear_model.ElasticNet(fit_intercept=False, alpha = self.reg, l1_ratio = self.l1_ratio, 
                                            tol = self.tol, max_iter = self.max_iter)

        model.fit(newX, newY)
        self.coef = model.coef_.T
        if Y.shape[1] == 1:
            self.coef = self.coef.reshape(-1,)
        if self.penalty is not None:
            self.converged = model.n_iter_ < self.max_iter
        else:
            self.converged = None
#         print X.shape
        self.dispersion = self.estimate_dispersion(X, Y, sample_weight)
        self.sd = self.estimate_sd(X, Y, sample_weight)
        self.ll = self.estimate_loglikelihood(sample_weight)

    def predict(self, X):
        """
        predict the Y value based on the model
        ----------
        X : design matrix
        Returns
        -------
        predicted value
        """
        if X.ndim == 1:
            X = X.reshape(-1,1)
        if self.fit_intercept:
            X = addIntercept(X)
        mu = np.dot(X, self.coef)
        return mu

    def log_probability(self, X, Y):
        """
        Given a set of X and Y, calculate the probability of
        observing Y value
        
        """

        if X.ndim == 1:
            X = X.reshape(-1,1)
        if self.fit_intercept:
            X = addIntercept(X)
        if Y.ndim == 1:
            Y = Y.reshape(-1,1)
#         print self.n_features
#         print X.shape
#         assert X.shape[1] == self.n_features
#         assert Y.shape[1] == self.n_targets
        
        pred = np.dot(X, self.coef)
        if pred.ndim == 1:
            pred = pred.reshape(-1,1)
        
        if Y.shape[1] == 1:
            if self.dispersion > 0:
                logP = (Y * pred - pred**2/2)/self.dispersion - Y**2/(2 * self.dispersion) - .5*np.log(2 * np.pi * self.dispersion)
                logP = logP.reshape(-1,)
            else:
#                 return None
                logP = np.zeros((Y.shape[0],))
                logP[Y.reshape(-1,)!=pred.reshape(-1,)] = -np.Infinity
                logP = logP.reshape(-1,)
        else:
            if np.linalg.det(self.dispersion) > 0:
                logP = -1/2*((Y.shape[1] * np.log(2 * np.pi) + np.log(np.linalg.det(self.dispersion))) +
                            np.diag(np.dot(np.dot(Y-pred, np.linalg.inv(self.dispersion)), (Y-pred).T)))
                logP = logP.reshape(-1,)
            else:
                if (np.diag(self.dispersion) > 0).all():
                    new_dispersion = np.diag(np.diag(self.dispersion))
                    logP = -1/2*((Y.shape[1] * np.log(2 * np.pi) + np.log(np.linalg.det(self.dispersion))) + 
                                 np.diag(np.dot(np.dot(Y-pred, np.linalg.inv(new_dispersion)), (Y-pred).T)))
                    logP = logP.reshape(-1,)

                else:
#                     return None
                    logP = np.zeros((Y.shape[0],))
                    logP[np.linalg.norm(Y-pred, axis = 1)!=0] = -np.Infinity
                    logP = logP.reshape(-1,)
        return logP
    
    
    def estimate_dispersion(self, X, Y, sample_weight):
        newX, newY = _rescale_data(X, Y, sample_weight)
        newPred = np.dot(newX, self.coef)
        if newPred.ndim == 1:
            newPred = newPred.reshape(-1,1)
        wresid = newY - newPred
        ssr = np.dot(wresid.T, wresid)
        sigma2 = ssr / np.sum(sample_weight)
        if sigma2.shape == (1,1):
            sigma2 = sigma2[0,0]
        return sigma2
        
        
    
    def estimate_sd(self, X, Y, sample_weight):
        newX, newY = _rescale_data(X, Y, sample_weight)
        if self.penalty is None:
            wX, wY = _rescale_data(X, Y, sample_weight ** 2)
            if newX.shape[1] == 1:
                try:
                    cov = 1 / np.dot(newX.T, newX)    
                    temp = np.dot(wX.T, wX) 
                    if newY.shape[1] == 1:
                        sd = np.sqrt(cov ** 2 * temp * self.dispersion).reshape(-1,)
                    else:
                        sd = np.sqrt(cov ** 2 * temp * np.diag(self.dispersion))
                except:
                    sd = None         
            else:
                try:
                    cov = np.linalg.inv(np.dot(newX.T, newX))
                    temp = np.dot(cov, wX.T)
                    if newY.shape[1] == 1:
                        sd = np.sqrt(np.diag(np.dot(temp,temp.T)) * self.dispersion).reshape(-1,)
                    else:
                        sd = np.sqrt(np.outer(np.diag(np.dot(temp,temp.T)), np.diag(self.dispersion)))
                except:
                    sd = None
        else:
            sd = None
        
        return sd

    
    def estimate_loglikelihood(self, sample_weight):
        q = self.n_targets
        sum_w = np.sum(sample_weight)
        if q == 1:
            if self.dispersion > 0:
                ll = - q * sum_w / 2 * np.log(2 * math.pi) - sum_w / 2 * np.log(self.dispersion) - q * sum_w / 2
            else:
                ll = None
        else:
            if np.linalg.det(self.dispersion) > 0:
                ll = - q * sum_w / 2 * np.log(2 * math.pi) - sum_w / 2 * np.log(np.linalg.det(self.dispersion)) - q * sum_w / 2
            else:
                if (np.diag(self.dispersion) > 0).all():
                    ll = - q * sum_w / 2 * np.log(2 * math.pi) - np.sum(sum_w / 2 * np.log(np.diag(self.dispersion))) - q * sum_w / 2
                else:
                    ll = None
        return ll

In [139]:
class MNL(BaseModel):
    """
    A MNL for data with input and output.
    """
        
    def fit(self, X, Y, sample_weight = None):
        """
        fit the weighted model
        Parameters
        ----------
        X : design matrix
        Y : response matrix
        sample_weight: sample weight vector

        """
        raise NotImplementedError

    def predict_probability(self, X):
        """
        predict the Y value based on the model
        ----------
        X : design matrix
        Returns
        -------
        predicted value
        """
        return np.exp(self.predict_log_probability(X))
    
    def predict_log_probability(self, X):
        """
        predict the Y value based on the model
        ----------
        X : design matrix
        Returns
        -------
        predicted value
        """

        if X.ndim == 1:
            X = X.reshape(-1,1)
        if self.fit_intercept:
            X = addIntercept(X)
#         print X
        p = np.dot(X, self.coef)
#         print p
        if p.ndim == 1:
            p = p.reshape(-1,1)
        p -= logsumexp(p, axis = 1)[:, np.newaxis]
        return p
    
    def predict(self, X):
        """
        predict the Y value based on the model
        ----------
        X : design matrix
        Returns
        -------
        predicted value
        """
        return NotImplementedError
        

    def log_probability(self, X, Y):
        """
        Given a set of X and Y, calculate the probability of
        observing Y value
        
        """
        
        return NotImplementedError
        
        
    
    def estimate_sd(self, X, sample_weight):
        if self.penalty == None:
            o_normalized = np.dot(X, self.coef)
            if o_normalized.ndim == 1:
                o_normalized = o_normalized.reshape(-1,1)
            o_normalized -= logsumexp(o_normalized, axis = 1)[:, np.newaxis]
            o_normalized = np.exp(o_normalized)
            # calculate hessian
            p = self.n_features
            q = self.n_targets
            h = np.zeros((p*(q-1), p*(q-1)))
            for e in range(q-1):
                for f in range(q-1):
                    h[e*p: (e+1)*p, f*p: (f+1)*p] = -np.dot(np.dot(X.T, 
                                                                  np.diag(np.multiply(np.multiply(o_normalized[:, f+1], 
                                                                                                  (e==f) - o_normalized[:, e+1]), 
                                                                                     sample_weight))), X)
            if np.sum(sample_weight) > 0:
                h = h / np.sum(sample_weight) * X.shape[0]
            if np.all(np.linalg.eigvals(-h) > 0):
                sd = np.sqrt(np.diag(np.linalg.inv(-h))).reshape(p,q-1, order = 'F')
                sd = np.hstack((np.zeros((p, 1)), sd))
            else:
                sd = None
        else:
            sd = None
        return sd

    
    def estimate_loglikelihood(self, X, Y, sample_weight):
        return NotImplementedError

In [140]:
class MNLD(MNL):
    """
    A MNL for discrete data with input and output.
    """
    def __init__(self, solver='newton-cg', fit_intercept = True, penalty = None, reg = 0, l1_ratio = 0, tol = 1e-4, max_iter = 100):
        super(MNLD, self).__init__(fam = 'MNLD', solver = solver, fit_intercept = fit_intercept, penalty = penalty, 
                                 reg = reg, l1_ratio = l1_ratio, tol = tol, max_iter = max_iter)
        
    def fit(self, X, Y, sample_weight = None):
        """
        fit the weighted model
        Parameters
        ----------
        X : design matrix
        Y : response matrix
        sample_weight: sample weight vector

        """

        if sample_weight is None:
            sample_weight = np.ones((X.shape[0],))
        assert Y.ndim == 1 or Y.shape[1] == 1
        assert X.shape[0] == Y.shape[0]
        assert X.shape[0] == sample_weight.shape[0]

        if self.reg == 0 or (self.penalty is None):
            penalty1 = 'l2'
            c = 1e200
        else:
            penalty1 = self.penalty
            c = 1/self.reg
        if X.ndim == 1:
            X = X.reshape(-1,1)
        if self.fit_intercept:
            X = addIntercept(X)
        self.n_samples = X.shape[0]
        self.n_features = X.shape[1]
        self.n_targets = len(np.unique(Y))
        if self.n_targets < 2:
            raise ValueError('n_targets < 2')
        
        self.lb = LabelBinarizer().fit(Y)

        model = linear_model.LogisticRegression(fit_intercept = False, penalty = penalty1, C = c,
                                                multi_class = 'multinomial', solver = self.solver,
                                                tol = self.tol, max_iter = self.max_iter)
#         print X.shape
#         print Y.shape
        Y_fit = self.lb.transform(Y)
        model.fit(X, Y, sample_weight = sample_weight)
        w0 = model.coef_
        if self.n_targets == 2:
            w0 = np.vstack((np.zeros((1, self.n_features)), w0*2))
#             w0 = np.vstack((w0,np.zeros((1, self.n_features))))
#         print w0.shape
#         print self.n_targets
        w1 = w0.reshape(self.n_targets, -1)
        w1 = w1.T - w1.T[:,0].reshape(-1,1)
        self.coef = w1
        self.converged = model.n_iter_ < self.max_iter
        self.sd = self.estimate_sd(X, sample_weight)
        self.ll = self.estimate_loglikelihood(X, Y, sample_weight)

    
    def predict(self, X):
        """
        predict the Y value based on the model
        ----------
        X : design matrix
        Returns
        -------
        predicted value
        """
        index = np.argmax(self.predict_log_probability(X), axis = 1)
        zero = np.zeros((X.shape[0], self.n_targets))
        zero[np.arange(X.shape[0]), index] = 1
        return self.lb.inverse_transform(zero)
#         return index
        

    def log_probability(self, X, Y):
        """
        Given a set of X and Y, calculate the probability of
        observing Y value
        
        """
        if X.ndim == 1:
            X = X.reshape(-1,1)
        assert Y.ndim == 1 or Y.shape[1] == 1
#         if self.fit_intercept:
#             assert X.shape[1] == self.n_features - 1
#         else:
#             assert X.shape[1] == self.n_features
        assert X.shape[0] == Y.shape[0]
        
        p = self.predict_log_probability(X)
        Y_transformed = self.lb.transform(Y)
        if Y_transformed.shape[1] == 1:
            Y_aug = np.zeros((X.shape[0],2))
            Y_aug[np.arange(X.shape[0]),Y_transformed.reshape(-1,)] = 1
        else:
            Y_aug = Y_transformed
        logP = np.sum(p*Y_aug, axis = 1)
        
        return logP
        
        
    
    def estimate_loglikelihood(self, X, Y, sample_weight):
        o_normalized_log = np.dot(X, self.coef)
        if o_normalized_log.ndim == 1:
            o_normalized_log = o_normalized_log.reshape(-1,1)
        o_normalized_log -= logsumexp(o_normalized_log, axis = 1)[:, np.newaxis]
        Y_aug = self.lb.transform(Y)
        ll = (sample_weight[:, np.newaxis] * Y_aug * o_normalized_log).sum()
        return ll

In [141]:
class MNLP(MNL):
    """
    A MNL with probability response for data with input and output.
    """
    def __init__(self, solver = 'newton-cg', fit_intercept = True, penalty = None, reg = 0, l1_ratio = 0, tol = 1e-4, max_iter = 100):
        super(MNL, self).__init__(fam = 'MNLP', solver = solver, fit_intercept = fit_intercept, penalty = penalty, 
                                 reg = reg, l1_ratio = l1_ratio, tol = tol, max_iter = max_iter)
        
    def fit(self, X, Y, sample_weight = None):
        """
        fit the weighted model
        Parameters
        ----------
        X : design matrix
        Y : response matrix
        sample_weight: sample weight vector

        """
        
        if sample_weight is None:
            sample_weight = np.ones((X.shape[0],))
        
        assert X.shape[0] == Y.shape[0]
        assert X.shape[0] == sample_weight.shape[0]
        
        
        if X.ndim == 1:
            X = X.reshape(-1,1)
        if self.fit_intercept:
            X = addIntercept(X)
        self.n_samples = X.shape[0]
        self.n_features = X.shape[1]
        self.n_targets = Y.shape[1]
        
        if self.n_targets < 2:
            raise ValueError('n_targets < 2')
        
        w0 = np.zeros((self.n_targets*self.n_features, ))

        if self.solver == 'lbfgs':
            func = lambda x, *args: _multinomial_loss_grad(x, *args)[0:2]
        else:
            func = lambda x, *args: _multinomial_loss(x, *args)[0]
            grad = lambda x, *args: _multinomial_loss_grad(x, *args)[1]
            hess = _multinomial_grad_hess

        if self.solver == 'lbfgs':
            try:
                w0, loss, info = optimize.fmin_l_bfgs_b(
                    func, w0, fprime=None,
                    args=(X, Y, self.reg, sample_weight),
                    iprint=0, pgtol=self.tol, maxiter=self.max_iter)
            except TypeError:
                # old scipy doesn't have maxiter
                w0, loss, info = optimize.fmin_l_bfgs_b(
                    func, w0, fprime=None,
                    args=(X, Y, self.reg, sample_weight),
                    iprint=0, pgtol=self.tol)
            if info["warnflag"] == 1:
                warnings.warn("lbfgs failed to converge. Increase the number "
                              "of iterations.")
            try:
                n_iter_i = info['nit'] - 1
            except:
                n_iter_i = info['funcalls'] - 1
        else:
            args = (X, Y, self.reg, sample_weight)
            w0, n_iter_i = newton_cg(hess, func, grad, w0, args=args,
                                             maxiter=self.max_iter, tol=self.tol)
            

    
  
        w1 = w0.reshape(self.n_targets, -1)
        w1 = w1.T - w1.T[:,0].reshape(-1,1)
        self.coef = w1
        self.converged = n_iter_i < self.max_iter
        self.sd = self.estimate_sd(X, sample_weight)
        self.ll = self.estimate_loglikelihood(X, Y, sample_weight)

 
    def predict(self, X):
        """
        predict the Y value based on the model
        ----------
        X : design matrix
        Returns
        -------
        predicted value
        """
        index = np.argmax(self.predict_log_probability(X), axis = 1)
        return index
        

    def log_probability(self, X, Y):
        """
        Given a set of X and Y, calculate the probability of
        observing Y value
        
        """
        
        if X.ndim == 1:
            X = X.reshape(-1,1)
        assert Y.ndim == 2
            
#         if self.fit_intercept:
#             assert X.shape[1] == self.n_features - 1
#         else:
#             assert X.shape[1] == self.n_features
        assert X.shape[0] == Y.shape[0]
        
        p = self.predict_log_probability(X)
        logP = np.sum(p*Y, axis = 1)
        
        return logP
        
        
    
    def estimate_loglikelihood(self, X, Y, sample_weight):
        o_normalized_log = np.dot(X, self.coef)
        if o_normalized_log.ndim == 1:
            o_normalized_log = o_normalized_log.reshape(-1,1)
        o_normalized_log -= logsumexp(o_normalized_log, axis = 1)[:, np.newaxis]
        ll = (sample_weight[:, np.newaxis] * Y * o_normalized_log).sum()
        return ll

In [142]:
def _multinomial_loss(w, X, Y, alpha, sample_weight):
    """Computes multinomial loss and class probabilities.
    Parameters
    ----------
    w : ndarray, shape (n_classes * n_features,) or
        (n_classes * (n_features + 1),)
        Coefficient vector.
    X : {array-like, sparse matrix}, shape (n_samples, n_features)
        Training data.
    Y : ndarray, shape (n_samples, n_classes)
        Transformed labels according to the output of LabelBinarizer.
    alpha : float
        Regularization parameter. alpha is equal to 1 / C.
    sample_weight : array-like, shape (n_samples,) optional
        Array of weights that are assigned to individual samples.
        If not provided, then each sample is given unit weight.
    Returns
    -------
    loss : float
        Multinomial loss.
    p : ndarray, shape (n_samples, n_classes)
        Estimated class probabilities.
    w : ndarray, shape (n_classes, n_features)
        Reshaped param vector excluding intercept terms.
    Reference
    ---------
    Bishop, C. M. (2006). Pattern recognition and machine learning.
    Springer. (Chapter 4.3.4)
    """
    n_classes = Y.shape[1]
    n_features = X.shape[1]

    w = w.reshape(n_classes, -1)
    sample_weight = sample_weight[:, np.newaxis]

    p = np.dot(X, w.T)
    p -= logsumexp(p, axis = 1)[:, np.newaxis]
    loss = -(sample_weight * Y * p).sum()
    loss += 0.5 * alpha * np.sum(w * w)
    p = np.exp(p, p)
    return loss, p, w
def _multinomial_loss_grad(w, X, Y, alpha, sample_weight):
    """Computes the multinomial loss, gradient and class probabilities.
    Parameters
    ----------
    w : ndarray, shape (n_classes * n_features,)
        Coefficient vector.
    X : {array-like, sparse matrix}, shape (n_samples, n_features)
        Training data.
    Y : ndarray, shape (n_samples, n_classes)
        Transformed labels according to the output of LabelBinarizer.
    alpha : float
        Regularization parameter. alpha is equal to 1 / C.
    sample_weight : array-like, shape (n_samples,) optional
        Array of weights that are assigned to individual samples.
    Returns
    -------
    loss : float
        Multinomial loss.
    grad : ndarray, shape (n_classes * n_features,) or
        (n_classes * (n_features + 1),)
        Ravelled gradient of the multinomial loss.
    p : ndarray, shape (n_samples, n_classes)
        Estimated class probabilities
    Reference
    ---------
    Bishop, C. M. (2006). Pattern recognition and machine learning.
    Springer. (Chapter 4.3.4)
    """
    n_classes = Y.shape[1]
    n_features = X.shape[1]
    grad = np.zeros((n_classes, n_features))
    loss, p, w = _multinomial_loss(w, X, Y, alpha, sample_weight)
    sample_weight = sample_weight[:, np.newaxis]
    diff = sample_weight * (p - Y)
    grad[:, :n_features] = np.dot(diff.T, X)
    grad[:, :n_features] += alpha * w
    return loss, grad.ravel(), p
def _multinomial_grad_hess(w, X, Y, alpha, sample_weight):
    """
    Computes the gradient and the Hessian, in the case of a multinomial loss.
    Parameters
    ----------
    w : ndarray, shape (n_classes * n_features,)
        Coefficient vector.
    X : {array-like, sparse matrix}, shape (n_samples, n_features)
        Training data.
    Y : ndarray, shape (n_samples, n_classes)
        Transformed labels according to the output of LabelBinarizer.
    alpha : float
        Regularization parameter. alpha is equal to 1 / C.
    sample_weight : array-like, shape (n_samples,) optional
        Array of weights that are assigned to individual samples.
    Returns
    -------
    grad : array, shape (n_classes * n_features,) or
        (n_classes * (n_features + 1),)
        Ravelled gradient of the multinomial loss.
    hessp : callable
        Function that takes in a vector input of shape (n_classes * n_features)
        or (n_classes * (n_features + 1)) and returns matrix-vector product
        with hessian.
    References
    ----------
    Barak A. Pearlmutter (1993). Fast Exact Multiplication by the Hessian.
        http://www.bcl.hamilton.ie/~barak/papers/nc-hessian.pdf
    """
    n_features = X.shape[1]
    n_classes = Y.shape[1]

    # `loss` is unused. Refactoring to avoid computing it does not
    # significantly speed up the computation and decreases readability
    loss, grad, p = _multinomial_loss_grad(w, X, Y, alpha, sample_weight)
    sample_weight = sample_weight[:, np.newaxis]

    # Hessian-vector product derived by applying the R-operator on the gradient
    # of the multinomial loss function.
    def hessp(v):
        v = v.reshape(n_classes, -1)
        # r_yhat holds the result of applying the R-operator on the multinomial
        # estimator.
        r_yhat = np.dot(X, v.T)
        r_yhat += (-p * r_yhat).sum(axis=1)[:, np.newaxis]
        r_yhat *= p
        r_yhat *= sample_weight
        hessProd = np.zeros((n_classes, n_features))
        hessProd[:, :n_features] = np.dot(r_yhat.T, X)
        hessProd[:, :n_features] += v * alpha
        return hessProd.ravel()

    return grad, hessp

# HMM

In [143]:
def calLogAlpha(log_prob_initial, log_prob_transition, log_Ey):

    ## return the alpha matrix which should be t * k
    # initial should be k * 1
    # transition prob shoud be (t - 1) * k * k
    # emission prob should be t * k * q
    # Ey should be t * k
    assert log_prob_initial.ndim == 1
    assert log_prob_transition.ndim == 3
    assert log_Ey.ndim == 2
    t = log_Ey.shape[0]
    k = log_Ey.shape[1]
    log_alpha = np.zeros((t, k))
    log_alpha[0, :] = log_prob_initial + log_Ey[0,:]
    for i in range(1, t):
        log_alpha[i, :] = logsumexp(log_prob_transition[i-1,:,:].T + log_alpha[i-1,:], axis = 1) + log_Ey[i,:]
    assert log_alpha.ndim == 2
    return log_alpha

def calLogBeta(log_prob_transition, log_Ey):

    ## return the alpha matrix which should be t * k
    # pi should be k * 1
    # transition prob shoud be (t - 1) * k * k
    # emission prob should be t * k * q
    # y should be t * k
    assert len(log_prob_transition.shape) == 3
    assert len(log_Ey.shape) == 2
    t = log_Ey.shape[0]
    k = log_Ey.shape[1]
    log_beta = np.zeros((t, k))
    for i in range(t-2, -1, -1):
        log_beta[i, :] = logsumexp(log_prob_transition[i, :, :] + (log_beta[i+1, :] + log_Ey[i+1,:]), axis = 1)
    assert len(log_beta.shape) == 2
    return log_beta

def calLogLikelihood(log_alpha):
    return logsumexp(log_alpha[-1,:])

def calLogGamma(log_alpha, log_beta, ll):
    return log_alpha + log_beta - ll

def calGamma(log_alpha, log_beta, ll):
    return np.exp(calLogGamma(log_alpha, log_beta, ll))

def calLogEpsilon(log_prob_transition, log_Ey, log_alpha, log_beta, ll):
    # epsilon should be t - 1 * k * k
    k = log_Ey.shape[1]
    return np.tile((log_Ey + log_beta)[1:,np.newaxis,:], [1,k,1]) + np.tile(log_alpha[:-1,:,np.newaxis], [1,1,k]) + log_prob_transition - ll

def calEpsilon(log_prob_transition, log_Ey, log_alpha, log_beta, ll):
    return np.exp(calLogEpsilon(log_prob_transition, log_Ey, log_alpha, log_beta, ll))

def calHMM(log_prob_initial, log_prob_transition, log_Ey):

    log_alpha = calLogAlpha(log_prob_initial, log_prob_transition, log_Ey)
    log_beta = calLogBeta(log_prob_transition, log_Ey)
    ll = calLogLikelihood(log_alpha)
    log_gamma = calLogGamma(log_alpha, log_beta, ll)
    log_epsilon = calLogEpsilon(log_prob_transition, log_Ey, log_alpha, log_beta, ll)
    return log_gamma, log_epsilon, ll


In [148]:
class SupervisedHMM:
    def __init__(self, num_states = 2, EM_tol = 1e-4, max_EM_iter = 100):
        self.num_states = num_states
        self.EM_tol = EM_tol
        self.max_EM_iter = max_EM_iter
        
    def setModels(self, model_emissions, model_initial = MNLP(), model_transition = MNLP()):
        # initial model and transition model must be MNLP
        self.model_initial = model_initial
        self.model_transition = [deepcopy(model_transition) for i in range(self.num_states)]
        self.model_emissions = [deepcopy(model_emissions) for i in range(self.num_states)]
        self.num_emissions = len(model_emissions)
    
    def setData(self, dfs):
        self.num_seqs = len(dfs)
        self.dfs = dfs
        
    
    def setInputs(self, covariates_initial, covariates_transition, covariates_emissions):
        # input should be a list inidicating the columns of the dataframe
        # 1. need to verify whether we need add intercept
        # 2. need to verify numpy array
        
        
#         self.inp_initials = [np.array(df.ix[0,covariates_initial]).reshape(1,-1).astype('float64') for df in self.dfs]
        self.inp_initials = [np.array(df[covariates_initial].iloc[0]).reshape(1,-1).astype('float64') for df in self.dfs]
        self.inp_initials_all_users = np.vstack(self.inp_initials)
        self.model_initial.coef = np.zeros((self.inp_initials_all_users.shape[1]+self.model_initial.fit_intercept,self.num_states))
        self.model_initial.coef = np.random.rand(self.inp_initials_all_users.shape[1]+self.model_initial.fit_intercept,self.num_states)
        
#         self.inp_transitions = [np.array(df.ix[1:, covariates_transition]).astype('float64') for df in self.dfs]
        self.inp_transitions = [np.array(df[covariates_transition].iloc[1:]).astype('float64') for df in self.dfs]
        self.inp_transitions_all_users = np.vstack(self.inp_transitions)
        
        for st in range(self.num_states):
            self.model_transition[st].coef = np.zeros((self.inp_transitions_all_users.shape[1]+self.model_transition[st].fit_intercept,self.num_states))
            self.model_transition[st].coef = np.random.rand(self.inp_transitions_all_users.shape[1]+self.model_transition[st].fit_intercept,self.num_states)
        self.inp_emissions = []
        self.inp_emissions_all_users = []
        for cov in covariates_emissions:
            self.inp_emissions.append([np.array(df[cov]).astype('float64') for df in self.dfs])
        for covs in self.inp_emissions:
            self.inp_emissions_all_users.append(np.vstack(covs))
        
        
    
    def setOutputs(self, responses_emissions):
        # output should be a list inidicating the columns of the dataframe
        self.out_emissions = []
        self.out_emissions_all_users = []
        for res in responses_emissions:
            self.out_emissions.append([np.array(df[res]) for df in self.dfs])
        for ress in self.out_emissions:
            self.out_emissions_all_users.append(np.vstack(ress))
        for i in range(self.num_states):
            for j in range(self.num_emissions):
                if isinstance(self.model_emissions[i][j], GLM):
                    self.model_emissions[i][j].coef = np.zeros((self.inp_emissions_all_users[j].shape[1]+self.model_emissions[i][j].fit_intercept,))
                    self.model_emissions[i][j].coef = np.random.rand(self.inp_emissions_all_users[j].shape[1]+self.model_emissions[i][j].fit_intercept,)
                    self.model_emissions[i][j].dispersion = 1
                if isinstance(self.model_emissions[i][j], LM):
                    if len(responses_emissions[j]) == 1:
                        self.model_emissions[i][j].coef = np.zeros((self.inp_emissions_all_users[j].shape[1]+self.model_emissions[i][j].fit_intercept,))
                        self.model_emissions[i][j].coef = np.random.rand(self.inp_emissions_all_users[j].shape[1]+self.model_emissions[i][j].fit_intercept,)
                        self.model_emissions[i][j].dispersion = 1
                    else:
                        self.model_emissions[i][j].coef = np.zeros((self.inp_emissions_all_users[j].shape[1]+self.model_emissions[i][j].fit_intercept, len(responses_emissions[j])))
                        self.model_emissions[i][j].coef = np.random.rand(self.inp_emissions_all_users[j].shape[1]+self.model_emissions[i][j].fit_intercept, len(responses_emissions[j]))
                        self.model_emissions[i][j].dispersion = np.eye(len(responses_emissions[j]))
                if isinstance(self.model_emissions[i][j], MNLD):
                    self.model_emissions[i][j].coef = np.zeros((self.inp_emissions_all_users[j].shape[1]+self.model_emissions[i][j].fit_intercept,np.unique(self.out_emissions_all_users[j]).shape[0]))
                    self.model_emissions[i][j].coef = np.random.rand(self.inp_emissions_all_users[j].shape[1]+self.model_emissions[i][j].fit_intercept,np.unique(self.out_emissions_all_users[j]).shape[0])
                    self.model_emissions[i][j].lb = LabelBinarizer().fit(self.out_emissions_all_users[j])
#                     self.model_emissions[i][j].n_targets = len(np.unique(self.out_emissions_all_users[j]))
                if isinstance(self.model_emissions[i][j], MNLP):
                    self.model_emissions[i][j].coef = np.zeros((self.inp_emissions_all_users[j].shape[1]+self.model_emissions[i][j].fit_intercept,len(responses_emissions[j])))
                    self.model_emissions[i][j].coef = np.random.rand(self.inp_emissions_all_users[j].shape[1]+self.model_emissions[i][j].fit_intercept,len(responses_emissions[j]))
    def EStep(self):
        self.log_gammas = []
        self.log_epsilons = []
        self.lls = []
        
        for seq in range(self.num_seqs):
            n_records = self.dfs[seq].shape[0]
            log_prob_initial = self.model_initial.predict_log_probability(self.inp_initials[seq]).reshape(self.num_states,)
            assert log_prob_initial.shape == (self.num_states,)
            # t - 1 * k * k 
            log_prob_transition = np.zeros((n_records - 1, self.num_states, self.num_states))
            for st in range(self.num_states):
                 log_prob_transition[:,st,:] = self.model_transition[st].predict_log_probability(self.inp_transitions[seq]) 
#             log_prob_transition = np.vstack([model.predict_log_probability(self.inp_transitions[seq]) for model in self.model_transition]).reshape(self.num_states, n_records-1, self.num_states)
#             log_prob_transition = np.swapaxes(log_prob_transition,0,1)
            assert log_prob_transition.shape == (n_records-1,self.num_states,self.num_states)
            
            log_Ey = np.zeros((n_records,self.num_states))
            for emis in range(self.num_emissions):
                model_collection = [models[emis] for models in self.model_emissions]
                log_Ey += np.vstack([model.log_probability(self.inp_emissions[emis][seq],
                                                           self.out_emissions[emis][seq]) for model in model_collection]).T

            
            log_gamma, log_epsilon, ll = calHMM(log_prob_initial, log_prob_transition, log_Ey)
            self.log_gammas.append(log_gamma)
            self.log_epsilons.append(log_epsilon)
            self.lls.append(ll)
            self.ll = sum(self.lls)

        
    def MStep(self):
        # optimize initial model
        X = self.inp_initials_all_users
        Y = np.exp(np.vstack([lg[0,:].reshape(1,-1) for lg in self.log_gammas]))
        logY = np.vstack([lg[0,:].reshape(1,-1) for lg in self.log_gammas])
        self.model_initial.fit(X, Y)
        
        # optimize transition models
        X = self.inp_transitions_all_users
        for st in range(self.num_states):
            Y = np.exp(np.vstack([eps[:,st,:] for eps in self.log_epsilons]))
            logY = np.vstack([eps[:,st,:] for eps in self.log_epsilons])
            self.model_transition[st].fit(X, Y)
        
        # optimize emission models
        for emis in range(self.num_emissions):
            X = self.inp_emissions_all_users[emis]
            Y = self.out_emissions_all_users[emis]
            for st in range(self.num_states):
                sample_weight = np.exp(np.hstack([lg[:,st] for lg in self.log_gammas]))
#                 if sum(sample_weight) ==0:
#                     'sum weight is 0'
#                     print np.hstack([lg[:,st] for lg in self.log_gammas])
#                     print sample_weight
#                 if len(sample_weight[sample_weight>0]==1):
#                     print np.hstack([lg[:,st] for lg in self.log_gammas])
#                     print sample_weight
                self.model_emissions[st][emis].fit(X, Y, sample_weight = sample_weight)
        
    
    def train(self):
        # maybe we should do some filtering here to filter out users with two few points.
        # this filter may be applied here or before we train.

        # set initial gamma and epsilons (to be equal likely).
        # this can be seen as the first E-Step.
#         a = np.random.rand(1,self.num_states)
#         a -= logsumexp(a, axis = 1)
#         self.log_gammas = [np.tile(a, [df.shape[0],1]) for df in self.dfs]
#         b = np.random.rand(self.num_states,self.num_states)
#         b -= logsumexp(b)
#         self.log_epsilons = [np.tile(b[np.newaxis,:,:], [df.shape[0]-1,1,1]) for i in range(self.num_seqs)]
#         self.ll = [-np.inf for i in range(self.num_seqs)]
        self.EStep()
        
        # start training
        for it in range(self.max_EM_iter):
#             print self.log_epsilons
            prev_ll = self.ll
            self.MStep()
            self.EStep()
#             print [self.model_transition[i].coef for i in range(self.num_states)]
            print self.ll
            if abs(self.ll-prev_ll) < self.EM_tol:
                break

        self.converged = it < self.max_EM_iter
    

## Tests of SupervisedHMM 

## Speed data

In [25]:
speed = pd.read_csv('speed.csv')
print speed.head()

   Unnamed: 0        rt corr  Pacc prev
0           1  6.456770  cor     0  inc
1           2  5.602119  cor     0  cor
2           3  6.253829  inc     0  cor
3           4  5.451038  inc     0  inc
4           5  5.872118  inc     0  inc


In [72]:
SHMM = SupervisedHMM(num_states=2, max_EM_iter=1000, EM_tol=1e-4)
SHMM.setData([speed])
SHMM.setModels(model_emissions = [LM()], model_transition=MNLP(solver='lbfgs'))
SHMM.setInputs(covariates_initial = [], covariates_transition = [], covariates_emissions = [[]])
SHMM.setOutputs([['rt']])

In [73]:
SHMM.train()

-305.268138632
-305.241632572
-305.203683475
-305.148388635
-305.062209716
-304.926214958
-304.679969726
-304.243782014
-303.351824605
-301.086020078
-292.76685388
-264.148200699
-176.576142995
-102.236217332
-92.6159593218
-92.4053029356
-92.4058061755
-92.4254256689
-92.4442018922
-92.4634562729
-92.4780725354
-92.4893997056
-92.4980134819
-92.5044792334
-92.5092928614
-92.5128575614
-92.5154881072
-92.5174246645
-92.5188479607
-92.5198928121
-92.52065921
-92.5212210296
-92.5216327043
-92.5219342667
-92.5221551195
-92.5223168374
-92.5224352401
-92.5225219218


In [74]:
print np.exp(SHMM.model_transition[0].coef - logsumexp(SHMM.model_transition[0].coef))
print np.exp(SHMM.model_transition[1].coef - logsumexp(SHMM.model_transition[1].coef))

[[ 0.91014227  0.08985773]]
[[ 0.19993659  0.80006341]]


In [75]:
print SHMM.model_emissions[0][0].coef
print SHMM.model_emissions[1][0].coef

[ 6.37957519]
[ 5.5013966]


In [76]:
print np.sqrt(SHMM.model_emissions[0][0].dispersion)
print np.sqrt(SHMM.model_emissions[1][0].dispersion)

0.247431398302
0.181899886497


In [174]:
SHMM = SupervisedHMM(num_states=2, max_EM_iter=1000, EM_tol=1e-4)
SHMM.setData([speed])
SHMM.setModels(model_emissions = [LM(), MNLD()], model_transition=MNLP(solver='lbfgs'))
SHMM.setInputs(covariates_initial = [], covariates_transition = [], covariates_emissions = [[],[]])
SHMM.setOutputs([['rt'],['corr']])

In [175]:
SHMM.train()

-550.594802353
-522.181893614
-415.305368632
-325.412145012
-308.284355193
-306.177101108
-305.638562576
-305.304622034
-305.08796374
-304.947607219
-304.861742505
-304.8096299
-304.777782838
-304.758066331
-304.745686357
-304.737814326
-304.732757882
-304.729485271
-304.727355648
-304.725964559
-304.725053534
-304.724455863
-304.724063306
-304.72380527
-304.723635571
-304.723523928
-304.723450464


In [176]:
print np.exp(SHMM.model_transition[0].coef - logsumexp(SHMM.model_transition[0].coef))
print np.exp(SHMM.model_transition[1].coef - logsumexp(SHMM.model_transition[1].coef))

[[ 0.90881076  0.09118924]]
[[ 0.19666679  0.80333321]]


In [177]:
print SHMM.model_emissions[0][0].coef
print SHMM.model_emissions[0][1].coef
print SHMM.model_emissions[1][0].coef
print SHMM.model_emissions[1][1].coef

[ 6.38719782]
[[ 0.         -2.18081328]]
[ 5.51213225]
[[ 0.         -0.10319339]]


In [237]:
SHMM = SupervisedHMM(num_states=2, max_EM_iter=1000, EM_tol=1e-4)
SHMM.setData([speed])
SHMM.setModels(model_emissions = [GLM(fam = family.Poisson()), MNLD()], model_transition=MNLP(solver='lbfgs'))
SHMM.setInputs(covariates_initial = [], covariates_transition = [], covariates_emissions = [[],[]])
SHMM.setOutputs([['rt_int'],['corr']])

In [238]:
SHMM.train()

-1045.82448828
-1045.67760078
-1045.51471704
-1045.31491643
-1045.08434346
-1044.73126144
-1044.36098677
-1043.81688361
-1043.08053845
-1041.49735439
-1040.63174451
-1039.99501723
-1039.47115581
-1039.059351
-1038.74665126
-1038.54032249
-1038.38460192
-1038.24264232
-1038.14918097
-1038.11083831
-1038.04860336
-1038.01240061
-1037.96226007
-1037.98714766
-1037.93685961
-1037.91821975
-1037.96019243
-1037.91821069
-1037.9066653
-1037.95493233
-1037.91493139
-1037.9058443
-1037.89171742
-1037.94754713
-1037.90054861
-1037.95457722
-1037.9163299
-1037.90877089
-1037.8958214
-1037.95250054
-1037.91640291
-1037.90944047
-1037.89674511
-1037.95409169
-1037.9176654
-1037.91072522
-1037.89806868
-1037.95547164
-1037.91884422
-1037.91185136
-1037.89919239
-1037.95647345
-1037.9197331
-1037.91268237
-1037.90001294
-1037.8848457
-1037.87145843
-1037.85791209
-1037.84526843
-1037.83397607
-1037.82136735
-1037.80977921
-1039.06196961
-1038.00097937
-1037.9645974
-1037.95890496
-1037.95772355
-1037

In [199]:
SHMM = SupervisedHMM(num_states=2, max_EM_iter=1000, EM_tol=1e-4)
SHMM.setData([speed])
SHMM.setModels(model_emissions = [MNLD()], model_transition=MNLP(solver='lbfgs'))
SHMM.setInputs(covariates_initial = [], covariates_transition = [], covariates_emissions = [[]])
SHMM.setOutputs([['corr']])

In [171]:
SHMM.train()

-249.482696414
-249.386640372
-249.304510389
-249.231574364
-249.163047803
-249.284318179
-249.216815871
-249.167376909
-249.113444861
-249.053512542
-248.98579058
-248.951704081
-248.789461652
-248.698963032
-248.645159658
-248.761429865
-248.666886524
-248.573716263
-248.706256008
-248.595057361
-248.485303981
-248.629712815
-248.49481264
-248.360955526
-248.698180639
-248.516311175
-248.38210355
-248.232589498
-248.622188523
-248.394103779
-248.224011055
-248.086149689
-248.513823566
-248.230425016
-248.01294718
-248.187676593
-247.919357356
-248.395084689
-248.040833101
-247.783863256
-248.274701534
-247.874776483
-248.026826303
-247.638481359
-247.639504897
-246.928824195
-246.992958444
-246.404221748
-245.964591168
-245.406675037
-244.917094801
-244.412491055
-243.970201417
-243.666390172
-243.372924216
-243.218614018
-243.537259176
-243.253589585
-243.171074479
-243.074012094
-243.038816472
-242.896093749
-242.801420126
-242.748344751
-242.697400146
-242.660617604
-242.632050165

In [39]:
speed[['corr_cor']] = (speed[['corr']] == 'cor')*1
speed[['corr_inc']] = (speed[['corr']] == 'inc')*1

In [90]:
SHMM = SupervisedHMM(num_states=2, max_EM_iter=1000, EM_tol=1e-4)
SHMM.setData([speed])
SHMM.setModels(model_emissions = [MNLP()], model_transition=MNLP(solver='lbfgs'))
SHMM.setInputs(covariates_initial = [], covariates_transition = [], covariates_emissions = [[]])
SHMM.setOutputs([['corr_cor', 'corr_inc']])

In [239]:
print np.exp(SHMM.model_transition[0].coef - logsumexp(SHMM.model_transition[0].coef))
print np.exp(SHMM.model_transition[1].coef - logsumexp(SHMM.model_transition[1].coef))

[[ 0.82657836  0.17342164]]
[[ 0.5  0.5]]


In [240]:
print SHMM.model_emissions[0][0].coef
print SHMM.model_emissions[1][0].coef
print SHMM.model_emissions[0][1].coef
print SHMM.model_emissions[1][1].coef

[ 1.73920482]
[ 1.66229956]
[[ 0.         -2.61160866]]
[[ 0.          1.39316674]]


In [42]:
SHMM = SupervisedHMM(num_states=2, max_EM_iter=1000, EM_tol=1e-4)
SHMM.setData([speed])
SHMM.setModels(model_emissions = [MNLP(), LM()], model_transition=MNLP(solver='lbfgs'))
SHMM.setInputs(covariates_initial = [], covariates_transition = [], covariates_emissions = [[],[]])
SHMM.setOutputs([['corr_cor', 'corr_inc'],['rt']])

In [43]:
SHMM.train()

-554.356505215
-554.266412655
-554.1568363
-554.004421166
-553.787395952
-553.463924056
-552.952769104
-552.088138392
-550.507553644
-547.365116065
-540.523964056
-524.421840281
-487.94724076
-409.021873913
-321.694639134
-302.425144008
-304.645575419
-304.630330904
-304.648839786
-304.666728735
-304.682945309
-304.695464385
-304.704451202
-304.710673101
-304.714896562
-304.717730763
-304.719619492
-304.720872697
-304.721701929
-304.722249648
-304.722611007
-304.722849237
-304.723006215
-304.72310962
-304.723177721


In [44]:
print np.exp(SHMM.model_transition[0].coef - logsumexp(SHMM.model_transition[0].coef))
print np.exp(SHMM.model_transition[1].coef - logsumexp(SHMM.model_transition[1].coef))

[[ 0.90881261  0.09118739]]
[[ 0.19666372  0.80333628]]


In [45]:
print SHMM.model_emissions[0][1].coef
print SHMM.model_emissions[1][1].coef

[ 6.38720339]
[ 5.51214045]


In [46]:
print SHMM.model_emissions[0][0].coef
print SHMM.model_emissions[1][0].coef

[[ 0.         -2.18085911]]
[[ 0.         -0.10319709]]


In [47]:
speed['rt_int'] = np.floor(speed['rt'])

In [53]:
SHMM = SupervisedHMM(num_states=2, max_EM_iter=1000, EM_tol=1e-4)
SHMM.setData([speed])
SHMM.setModels(model_emissions = [GLM(fam = family.Poisson())], model_transition=MNLP(solver='lbfgs'))
SHMM.setInputs(covariates_initial = [], covariates_transition = [], covariates_emissions = [[]])
SHMM.setOutputs([['rt_int']])

In [54]:
SHMM.train()

-797.004237043
-797.003797161
-797.003796486
